In [ ]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt

# Load the hyperspectral data
hyperspectral_data = scipy.io.loadmat(r"C:\Users\Soham\Downloads\Hyper1.mat")
hyperspectral_data = hyperspectral_data['img']  # Ensure the key is correct
height, width, bands = hyperspectral_data.shape

# Fixing the filtering issue (selecting all bands for now)
# To filter specific bands, use indexing or other criteria.
filtered_data = hyperspectral_data  # No filtering for now

print(f"Shape after filtering: {filtered_data.shape}")

# Ensure the band index is valid
band_index = 10  # Change to a valid index between 0 and bands-1
if band_index < 0 or band_index >= bands:
    raise ValueError(f"Invalid band index: {band_index}. Must be between 0 and {bands-1}.")

# Extract and display a specific band
band = filtered_data[:, :, band_index]
plt.imshow(band)
plt.colorbar()
plt.title(f'Filtered Hyperspectral Image - Band {band_index}')
plt.show()


In [ ]:
#Have to run
import torch
import torch.nn as nn
import torch.optim as optim

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


input_dim = 224  # Background feature size
#have to try reducing this 
latent_dim = 64  # Latent vector size 
batch_size = 64
epochs = 50
#try 
lr = 0.0002

# Encoder
class Encoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(Encoder, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, latent_dim)
        )

    def forward(self, x):
        return self.net(x)

# Decoder
class Decoder(nn.Module):
    def __init__(self, latent_dim, output_dim):
        super(Decoder, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim)
        )

    def forward(self, x):
        return self.net(x)

class LatentDiscriminator(nn.Module):
    def __init__(self, latent_dim):
        super(LatentDiscriminator, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

# Discriminator for reconstructed input
class ReconstructionDiscriminator(nn.Module):
    def __init__(self, input_dim):
        super(ReconstructionDiscriminator, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

# Initialize models
encoder = Encoder(input_dim, latent_dim).to(device)
decoder = Decoder(latent_dim, input_dim).to(device)
latent_discriminator = LatentDiscriminator(latent_dim).to(device)
reconstruction_discriminator = ReconstructionDiscriminator(input_dim).to(device)

mse_loss = nn.MSELoss()
bce_loss = nn.BCELoss()

opt_encoder_decoder = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=lr)
opt_latent_discriminator = optim.Adam(latent_discriminator.parameters(), lr=lr)
opt_reconstruction_discriminator = optim.Adam(reconstruction_discriminator.parameters(), lr=lr)



In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

# Load the encoder and decoder models
encoder.load_state_dict(torch.load("encoder_new.pth"))
decoder.load_state_dict(torch.load("decoder_new.pth"))
encoder.to(device)
decoder.to(device)


# Assuming DTest is your hyperspectral test data with shape (400, 400, 189)
# Reshape the test data to be (400*400, 189) for feeding into the model
test_data = filtered_data.reshape(-1, 224)  # Shape will be (160000, 189) (flatten each pixel)

# Convert the reshaped data to a tensor and move it to the appropriate device
test_data_tensor = torch.tensor(test_data, dtype=torch.float32).to(device)

# Pass the test data through the encoder and decoder
with torch.no_grad():
    z_test = encoder(test_data_tensor)  # Get the encoded representation
    x_reconstructed = decoder(z_test)  # Reconstruct the hyperspectral data

# Compute the reconstruction error (Mean Squared Error) per pixel
reconstruction_error = torch.mean((test_data_tensor - x_reconstructed) ** 2, dim=1)

# Set a threshold for anomaly detection (e.g., 95th percentile of the reconstruction error)
threshold = torch.quantile(reconstruction_error, 0.95)

# Detect anomalies (where reconstruction error exceeds the threshold)
anomalies = reconstruction_error > threshold

# Reshape the anomalies to match the spatial dimensions of the image (400, 400)
reshaped_anomalies = anomalies.cpu().numpy().reshape(2085, 682)

# Visualize the anomalies in the hyperspectral image
plt.figure(figsize=(10, 8))
plt.imshow(reshaped_anomalies, cmap='hot', interpolation='nearest')
plt.colorbar()
plt.title('Detected Anomalies in Hyperspectral Image')
plt.show()


In [ ]:
#supression by min size

import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import label, find_objects

# Load your anomaly map (e.g., 'reshaped_anomalies' from the output above)
anomaly_map = reshaped_anomalies  # Assuming this is your binary map (400x400)

# Label connected components
labeled_map, num_features = label(anomaly_map)

# Filter out small components
size_threshold = 30  # Adjust this threshold based on the size of your plane
filtered_map = np.zeros_like(labeled_map)

for i in range(1, num_features + 1):
    region = (labeled_map == i)
    if np.sum(region) >= size_threshold:
        filtered_map[region] = 1

# Visualize the filtered anomaly map
plt.figure(figsize=(10, 8))
plt.imshow(filtered_map, cmap='hot', interpolation='nearest')
plt.colorbar()
plt.title('Filtered Anomalies (Large Structures Only)')
plt.show()


In [ ]:
import numpy as np

# Example maps

# Total elements (size of the map)
total_elements = map.size

# Correct matches (where filtered_map matches map)
matching_elements = np.sum(filtered_map == map)

# Calculate accuracy
accuracy = (matching_elements / total_elements) * 100

# Display the result
print(f"Accuracy: {accuracy:.2f}%")


In [ ]:
#ye gaussian blur wala hai,check kar agar op close hai kya koi ek use karege,sigma and threshold change karo 

import numpy as np
import matplotlib.pyplot as plt
from skimage.measure import label, regionprops
from skimage.morphology import remove_small_objects
from scipy.ndimage import gaussian_filter

def refine_connected_components(binary_anomaly_map, size_threshold=50):
    """
    Refines anomaly map by removing small connected components based on size threshold.
    """
    labeled_map = label(binary_anomaly_map)
    refined_map = np.zeros_like(binary_anomaly_map)

    for region in regionprops(labeled_map):
        if region.area >= size_threshold:
            for coords in region.coords:
                refined_map[coords[0], coords[1]] = 1

    return refined_map

# Assuming filtered_anomaly_map and map are defined (map = ground truth)

# Test size_thresholds from 10 to 100 in steps of 10
best_accuracy = 0
best_threshold = 0
results = {}

for size_threshold in range(10, 110, 10):
    # Step 1: Refine Anomaly Map
    refined_map = refine_connected_components(binary_anomaly_map, size_threshold=size_threshold)
    
    # Step 2: Optional Smoothing and Re-thresholding
    sigma = 2
    smoothed_map = gaussian_filter(refined_map.astype(float), sigma=sigma)
    threshold = 0.3
    final_map = (smoothed_map > threshold).astype(int)
    
    # Step 3: Compute Metrics
    TP = np.sum((final_map == 1) & (map == 1))  # True Positives
    TN = np.sum((final_map == 0) & (map == 0))  # True Negatives
    FP = np.sum((final_map == 1) & (map == 0))  # False Positives
    FN = np.sum((final_map == 0) & (map == 1))  # False Negatives

    accuracy = (TP + TN) / (TP + TN + FP + FN)
    results[size_threshold] = accuracy

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_threshold = size_threshold

    print(f"Size Threshold: {size_threshold}, Accuracy: {accuracy:.4f}")

# Display the best results
print("\nBest Results:")
print(f"Best Size Threshold: {best_threshold}")
print(f"Best Accuracy: {best_accuracy:.4f}")

# Visualize the best anomaly map
best_refined_map = refine_connected_components(filtered_anomaly_map, size_threshold=best_threshold)
smoothed_best_map = gaussian_filter(best_refined_map.astype(float), sigma=2)
final_best_map = (smoothed_best_map > 0.5).astype(int)

plt.figure(figsize=(12, 10))
plt.imshow(final_best_map, cmap='gray')
plt.title(f'Refined Anomaly Map (Best Threshold = {best_threshold})')
plt.xlabel('Pixel X-axis')
plt.ylabel('Pixel Y-axis')
plt.show()
